In [2]:
import pandas as pd

# 读取CSV文件
spy_growth = pd.read_csv('/usr3/graduate/xz0224/normal/spyg.csv')
spy_value = pd.read_csv('/usr3/graduate/xz0224/normal/spyv.csv')
sp500 = pd.read_csv('/usr3/graduate/xz0224/normal/spy.csv')
cash = pd.read_csv('/usr3/graduate/xz0224/normal/cash.csv')
qqq_cc = pd.read_csv('/usr3/graduate/xz0224/normal/qqq.csv')
qqq_co = pd.read_csv('/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qqq.csv')

# 转换日期列为日期时间格式
spy_growth['date'] = pd.to_datetime(spy_growth['date'])
spy_value['date'] = pd.to_datetime(spy_value['date'])
sp500['date'] = pd.to_datetime(sp500['date'])
cash['date'] = pd.to_datetime(cash['date'])
qqq_cc['date'] = pd.to_datetime(qqq_cc['date'])
qqq_co['date'] = pd.to_datetime(qqq_co['date'])

# 初始化投资金额
initial_investment = 100

# Oracle 对于 Growth 和 Value 的选择
oracle_values_GV = [initial_investment]
oracle_dates_GV = [pd.to_datetime('2000-01-01')]

# 模拟 Oracle 投资过程
# current_value = initial_investment
# for date in pd.date_range(start='2000-01-01', end='2024-06-30'):
#     if date in spy_growth['date'].values and date in spy_value['date'].values:
#         growth_rate = spy_growth.loc[spy_growth['date'] == date, 'return'].values[0]
#         value_rate = spy_value.loc[spy_value['date'] == date, 'return'].values[0]
#         # Oracle 选择收益率更高的投资
#         current_value *= (1 + max(growth_rate, value_rate))
#         oracle_values_GV.append(current_value)
#         oracle_dates_GV.append(date)

# print(f"Final Oracle investment amount for Growth vs Value: {current_value}")

# 'Buy and Hold' 策略的投资值
buy_hold_growth_values = [initial_investment]
buy_hold_value_values = [initial_investment]
buy_hold_spy_values = [initial_investment]
buy_hold_qqq_cc_values = [initial_investment]
buy_hold_qqq_co_values = [initial_investment]

# 初始化日期列表
buy_hold_dates = [pd.to_datetime('2000-01-01')]

# 遍历每个日期，计算投资价值
for date in pd.date_range(start='2000-01-01', end='2024-06-30'):
    if date in spy_growth['date'].values and date in spy_value['date'].values and date in sp500['date'].values and date in qqq_cc['date'].values and date in qqq_co['date'].values:
        growth_rate = spy_growth.loc[spy_growth['date'] == date, 'return'].values[0]
        value_rate = spy_value.loc[spy_value['date'] == date, 'return'].values[0]
        spy_rate = sp500.loc[sp500['date'] == date, 'return'].values[0]
        qqq_cc_rate = qqq_cc.loc[qqq_cc['date'] == date, 'return'].values[0]
        qqq_co_rate = qqq_co.loc[qqq_co['date'] == date, 'return'].values[0]

        # 计算 'Buy and Hold' 策略的投资价值
        current_value_growth = buy_hold_growth_values[-1] * (1 + growth_rate)
        current_value_value = buy_hold_value_values[-1] * (1 + value_rate)
        current_value_spy = buy_hold_spy_values[-1] * (1 + spy_rate)
        current_value_qqq_cc = buy_hold_qqq_cc_values[-1] * (1 + qqq_cc_rate)
        current_value_qqq_co = buy_hold_qqq_co_values[-1] * (1 + qqq_co_rate)

        buy_hold_growth_values.append(current_value_growth)
        buy_hold_value_values.append(current_value_value)
        buy_hold_spy_values.append(current_value_spy)
        buy_hold_qqq_cc_values.append(current_value_qqq_cc)
        buy_hold_qqq_co_values.append(current_value_qqq_co)
        buy_hold_dates.append(date)

# # Oracle 对于 SPY 和 Cash 的选择
# oracle_values_MC = [initial_investment]
# oracle_dates_MC = [pd.to_datetime('2000-01-01')]

# # 模拟 Oracle 投资过程
# current_value = initial_investment
# for date in pd.date_range(start='2000-01-01', end='2024-06-30'):
#     if date in sp500['date'].values:
#         spy_rate = sp500.loc[sp500['date'] == date, 'return'].values[0]
#         if spy_rate > 0:
#             current_value *= (1 + spy_rate)
#         else:
#             # 如果SPY的回报率为负，则保持现金不变
#             current_value *= 1
#         oracle_values_MC.append(current_value)
#         oracle_dates_MC.append(date)


In [4]:
import pandas as pd
import empyrical
import os


# 读取Excel文件中的数据路径
file_paths = {
    'MC-1Y-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'MC-6M-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'MC-1Y-CC': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'MC-1Y-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'MC-6M-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'MC-3M-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_3months_c-c).xlsx',
    'QQQ-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/investment_results_final(2006)(train_1years_c-c).xlsx',
    'QQ-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_data-2.xlsx',
    'QQ-CC': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx'
}

strategies_data = {
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values),
    'S\&P': (buy_hold_dates, buy_hold_spy_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)


# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Create the directory if it does not exist
os.makedirs('jul3', exist_ok=True)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul3/end_year_values-jul3.xlsx', index=False)
annual_returns_df.to_excel('jul3/annual_returns-jul3.xlsx', index=False)
max_drawdowns_df.to_excel('jul3/max_drawdowns-jul3.xlsx', index=False)
annual_volatility_df.to_excel('jul3/annual_volatility-jul3.xlsx', index=False)
sharpe_ratios_df.to_excel('jul3/sharpe_ratios-jul3.xlsx', index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx'

In [7]:
import pandas as pd
import empyrical
import os


# 读取Excel文件中的数据路径
file_paths = {
    'MC-1Y-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'MC-1Y-CC': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'MC-1Y-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'MC-6M-OC': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'MC-6M-CC': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'MC-3M-CC': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_3months_c-c).xlsx',
}

strategies_data = {
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values),
    'S\&P': (buy_hold_dates, buy_hold_spy_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)


# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Create the directory if it does not exist
os.makedirs('jul4', exist_ok=True)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul4/end_year_values-jul4.xlsx', index=False)
annual_returns_df.to_excel('jul4/annual_returns-jul4.xlsx', index=False)
max_drawdowns_df.to_excel('jul4/max_drawdowns-jul4.xlsx', index=False)
annual_volatility_df.to_excel('jul4/annual_volatility-jul4.xlsx', index=False)
sharpe_ratios_df.to_excel('jul4/sharpe_ratios-jul4.xlsx', index=False)


/scratch/ipykernel_3737161/269470830.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3737161/269470830.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3737161/269470830.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3737161/269470830.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3737161/269470830.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3737161/269470830.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instea

In [9]:
import pandas as pd
import empyrical
import os


# 读取Excel文件中的数据路径
file_paths = {
    'M1Y-Q3M': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_3m_mc_1y.xlsx',
    'M1Y-Q1Y': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_1y_mc_1y.xlsx',
    'MC-1Y-CMB': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_mc_1y_mc_1y.xlsx',
    'CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CC': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx',  
    'CC': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_3months_c-c).xlsx',  
    'CC': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_3months_c-c).xlsx',
    'OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_3months_c-c).xlsx'
}

strategies_data = {
    'S\&P': (buy_hold_dates, buy_hold_spy_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)


# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df','sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df



# Create the directory if it does not exist
os.makedirs('jul6-mc-新内容', exist_ok=True)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul6-mc-新内容/end_year_values-jul6-mc-新内容.xlsx', index=False)
annual_returns_df.to_excel('jul6-mc-新内容/annual_returns-jul6-mc-新内容.xlsx', index=False)
max_drawdowns_df.to_excel('jul6-mc-新内容/max_drawdowns-jul6-mc-新内容.xlsx', index=False)
annual_volatility_df.to_excel('jul6-mc-新内容/annual_volatility-jul6-mc-新内容.xlsx', index=False)
sharpe_ratios_df.to_excel('jul6-mc-新内容/sharpe_ratios-jul6-mc-新内容.xlsx', index=False)

/scratch/ipykernel_2478019/839184847.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/839184847.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/839184847.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/839184847.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/839184847.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/839184847.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instea

In [15]:
import pandas as pd
import empyrical
import os


# 读取Excel文件中的数据路径
file_paths = {
    'QQ-1Y-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_1years_c-c).xlsx',
    'QQ-1Y-CC': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'QQ-1Y-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/investment_results_final(2006)(train_1years_c-c).xlsx',
    'QQ-6M-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 6月很高.xlsx',  
    'QQ-6M-CC': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'QQ-6M-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'QQ-3M-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',  
    'QQ-3M-CC': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'QQ-3M-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
}

strategies_data = {
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)


# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

import os


# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df','sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df
# Create the directory if it does not exist
os.makedirs('jul5-qq-变化过', exist_ok=True)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul5-qq-变化过/end_year_values-jul5-qq-变化过.xlsx', index=False)
annual_returns_df.to_excel('jul5-qq-变化过/annual_returns-jul5-qq-变化过.xlsx', index=False)
max_drawdowns_df.to_excel('jul5-qq-变化过/max_drawdowns-jul5-qq-变化过.xlsx', index=False)
annual_volatility_df.to_excel('jul5-qq-变化过/annual_volatility-jul5-qq-变化过.xlsx', index=False)
sharpe_ratios_df.to_excel('jul5-qq-变化过/sharpe_ratios-jul5-qq-变化过.xlsx', index=False)


/scratch/ipykernel_2216022/3355240254.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2216022/3355240254.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2216022/3355240254.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2216022/3355240254.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2216022/3355240254.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2216022/3355240254.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [2]:
import pandas as pd

# 读取Excel文件
file_path_close_open = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx'
file_path_open_close = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'

data_close_open = pd.read_excel(file_path_close_open)
data_open_close = pd.read_excel(file_path_open_close)

# 确保日期列格式为datetime
data_close_open['Date'] = pd.to_datetime(data_close_open['Date'])
data_open_close['Date'] = pd.to_datetime(data_open_close['Date'])

# 选择所需的列
data_close_open = data_close_open[['Date', 'Investment_Value']]
data_open_close = data_open_close[['Date', 'Investment_Value']]

# 确保日期对齐，合并两个数据集
merged_data = pd.merge(data_close_open, data_open_close, on='Date', how='inner', suffixes=('_close_open', '_open_close'))

# 计算策略融合的值
merged_data['Investment_Value'] = (merged_data['Investment_Value_close_open'] * merged_data['Investment_Value_open_close']) / 100

# 保持原列名并添加融合策略值
final_data = merged_data[['Date', 'Investment_Value', 'Investment_Value_close_open', 'Investment_Value_open_close']]

# 保存结果到新的Excel文件
output_file_path = '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_3m_mc_1y.xlsx'
final_data.to_excel(output_file_path, index=False)

print(f"策略融合的结果已保存到 {output_file_path}")


策略融合的结果已保存到 /usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_3m_mc_1y.xlsx


In [8]:
import pandas as pd

# 读取Excel文件
file_path_close_open = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx'
file_path_open_close = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/investment_results_final(2006)(train_1years_c-c).xlsx'

data_close_open = pd.read_excel(file_path_close_open)
data_open_close = pd.read_excel(file_path_open_close)

# 确保日期列格式为datetime
data_close_open['Date'] = pd.to_datetime(data_close_open['Date'])
data_open_close['Date'] = pd.to_datetime(data_open_close['Date'])

# 选择所需的列
data_close_open = data_close_open[['Date', 'Investment_Value']]
data_open_close = data_open_close[['Date', 'Investment_Value']]

# 确保日期对齐，合并两个数据集
merged_data = pd.merge(data_close_open, data_open_close, on='Date', how='inner', suffixes=('_close_open', '_open_close'))

# 计算策略融合的值
merged_data['Investment_Value'] = (merged_data['Investment_Value_close_open'] * merged_data['Investment_Value_open_close']) / 100

# 保持原列名并添加融合策略值
final_data = merged_data[['Date', 'Investment_Value', 'Investment_Value_close_open', 'Investment_Value_open_close']]

# 保存结果到新的Excel文件
output_file_path = '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_1y_mc_1y.xlsx'
final_data.to_excel(output_file_path, index=False)

print(f"策略融合的结果已保存到 {output_file_path}")


策略融合的结果已保存到 /usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_1y_mc_1y.xlsx


In [14]:
import pandas as pd

# 读取Excel文件
file_path_close_open = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'
file_path_open_close = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'

data_close_open = pd.read_excel(file_path_close_open)
data_open_close = pd.read_excel(file_path_open_close)

# 确保日期列格式为datetime
data_close_open['Date'] = pd.to_datetime(data_close_open['Date'])
data_open_close['Date'] = pd.to_datetime(data_open_close['Date'])

# 选择所需的列
data_close_open = data_close_open[['Date', 'Investment_Value']]
data_open_close = data_open_close[['Date', 'Investment_Value']]

# 确保日期对齐，合并两个数据集
merged_data = pd.merge(data_close_open, data_open_close, on='Date', how='inner', suffixes=('_close_open', '_open_close'))

# 计算策略融合的值
merged_data['Investment_Value'] = (merged_data['Investment_Value_close_open'] * merged_data['Investment_Value_open_close']) / 100

# 保持原列名并添加融合策略值
final_data = merged_data[['Date', 'Investment_Value', 'Investment_Value_close_open', 'Investment_Value_open_close']]

# 保存结果到新的Excel文件
output_file_path = '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_mc_1y_mc_1y.xlsx'
final_data.to_excel(output_file_path, index=False)

print(f"策略融合的结果已保存到 {output_file_path}")


策略融合的结果已保存到 /usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_mc_1y_mc_1y.xlsx


In [16]:
import pandas as pd
import empyrical
import os


# 读取Excel文件中的数据路径
file_paths = {
    'MC-1Y-100-$\\alpha=0.001$': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx',
    'QQ-1Y-200-$\\alpha=0.01$': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_1years_c-c).xlsx', 
}

strategies_data = {
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)


# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df','sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df
import os

# Create the directory if it does not exist
os.makedirs('加强', exist_ok=True)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('加强/end_year_values-加强.xlsx', index=False)
annual_returns_df.to_excel('加强/annual_returns-加强.xlsx', index=False)
max_drawdowns_df.to_excel('加强/max_drawdowns-加强.xlsx', index=False)
annual_volatility_df.to_excel('加强/annual_volatility-加强.xlsx', index=False)
sharpe_ratios_df.to_excel('加强/sharpe_ratios-加强.xlsx', index=False)



/scratch/ipykernel_2216022/3910716329.py:39: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2216022/3910716329.py:39: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')


In [2]:
import pandas as pd

# Define file paths
mc_co_file_path = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'
mc_oc_file_path = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'
mc_cc_file_path = '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx'
qq_co_file_path = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx'
qq_cc_file_path = '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx'

# Load data
mc_co_data = pd.read_excel(mc_co_file_path)
mc_oc_data = pd.read_excel(mc_oc_file_path)
mc_cc_data = pd.read_excel(mc_cc_file_path)
qq_co_data = pd.read_excel(qq_co_file_path)
qq_cc_data = pd.read_excel(qq_cc_file_path)

# Merge mc_co and mc_oc strategies
merged_mc_data = pd.merge(mc_oc_data, mc_co_data, on='Date', suffixes=('_oc', '_co'))

# Initialize variables to keep track of the current state and transaction count for merged mc strategy
current_state = 'cash'
transaction_count = 0
transaction_log = []

# Simulate the combined mc strategy
for index, row in merged_mc_data.iterrows():
    date = row['Date']
    decision_oc = row['Decision_oc']
    decision_co = row['Decision_co']

    # First perform the oc transaction
    if current_state != decision_oc:
        transaction_count += 1
        transaction_log.append((date, 'oc', current_state, decision_oc))
        current_state = decision_oc

    # Then perform the co transaction
    if current_state != decision_co:
        transaction_count += 1
        transaction_log.append((date, 'co', current_state, decision_co))
        current_state = decision_co

# Create a DataFrame to display the transaction log
merged_mc_transaction_log_df = pd.DataFrame(transaction_log, columns=['Date', 'Strategy', 'From', 'To'])

# Calculate the transaction cost for mc_cc, qq_cc, qq_co, mc_co, and mc_oc strategies
def calculate_transaction_count(data):
    current_state = 'cash'
    transaction_count = 0
    transaction_log = []

    for index, row in data.iterrows():
        date = row['Date']
        decision = row['Decision']

        if current_state != decision:
            transaction_count += 1
            transaction_log.append((date, current_state, decision))
            current_state = decision

    return pd.DataFrame(transaction_log, columns=['Date', 'From', 'To'])

mc_cc_transaction_log_df = calculate_transaction_count(mc_cc_data)
qq_cc_transaction_log_df = calculate_transaction_count(qq_cc_data)
qq_co_transaction_log_df = calculate_transaction_count(qq_co_data)
mc_co_transaction_log_df = calculate_transaction_count(mc_co_data)
mc_oc_transaction_log_df = calculate_transaction_count(mc_oc_data)

# Calculate annual transaction count for each strategy
def calculate_annual_transaction_count(transaction_log_df, start_year, end_year):
    transaction_log_df['Year'] = pd.to_datetime(transaction_log_df['Date']).dt.year
    annual_transaction_count = transaction_log_df.groupby('Year').size().reindex(range(start_year, end_year+1), fill_value=0).reset_index(name='Transaction_Count')
    return annual_transaction_count

start_year = min(
    mc_cc_transaction_log_df['Date'].dt.year.min(),
    qq_cc_transaction_log_df['Date'].dt.year.min(),
    qq_co_transaction_log_df['Date'].dt.year.min(),
    mc_co_transaction_log_df['Date'].dt.year.min(),
    mc_oc_transaction_log_df['Date'].dt.year.min(),
    merged_mc_transaction_log_df['Date'].dt.year.min()
)

end_year = max(
    mc_cc_transaction_log_df['Date'].dt.year.max(),
    qq_cc_transaction_log_df['Date'].dt.year.max(),
    qq_co_transaction_log_df['Date'].dt.year.max(),
    mc_co_transaction_log_df['Date'].dt.year.max(),
    mc_oc_transaction_log_df['Date'].dt.year.max(),
    merged_mc_transaction_log_df['Date'].dt.year.max()
)

merged_mc_annual_transaction_count = calculate_annual_transaction_count(merged_mc_transaction_log_df, start_year, end_year)
mc_cc_annual_transaction_count = calculate_annual_transaction_count(mc_cc_transaction_log_df, start_year, end_year)
qq_cc_annual_transaction_count = calculate_annual_transaction_count(qq_cc_transaction_log_df, start_year, end_year)
qq_co_annual_transaction_count = calculate_annual_transaction_count(qq_co_transaction_log_df, start_year, end_year)
mc_co_annual_transaction_count = calculate_annual_transaction_count(mc_co_transaction_log_df, start_year, end_year)
mc_oc_annual_transaction_count = calculate_annual_transaction_count(mc_oc_transaction_log_df, start_year, end_year)

# Add statistical analysis
def add_statistical_analysis(df):
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()
    return pd.DataFrame({
        'Max': max_values,
        'Min': min_values,
        'Median': median_values,
        'Mean': average_values,
        'StdDev': stddev_values
    })

merged_mc_stats = add_statistical_analysis(merged_mc_annual_transaction_count)
mc_cc_stats = add_statistical_analysis(mc_cc_annual_transaction_count)
qq_cc_stats = add_statistical_analysis(qq_cc_annual_transaction_count)
qq_co_stats = add_statistical_analysis(qq_co_annual_transaction_count)
mc_co_stats = add_statistical_analysis(mc_co_annual_transaction_count)
mc_oc_stats = add_statistical_analysis(mc_oc_annual_transaction_count)

# Combine annual transaction count and statistical analysis into a single DataFrame
def combine_annual_and_stats(annual_df, stats_df, strategy_name):
    combined_df = annual_df.copy()
    stats_df.reset_index(inplace=True)
    combined_df = pd.concat([combined_df, stats_df], axis=1)
    combined_df['Strategy'] = strategy_name
    return combined_df

merged_mc_combined = combine_annual_and_stats(merged_mc_annual_transaction_count, merged_mc_stats, 'Merged_MC')
mc_cc_combined = combine_annual_and_stats(mc_cc_annual_transaction_count, mc_cc_stats, 'MC_CC')
qq_cc_combined = combine_annual_and_stats(qq_cc_annual_transaction_count, qq_cc_stats, 'QQ_CC')
qq_co_combined = combine_annual_and_stats(qq_co_annual_transaction_count, qq_co_stats, 'QQ_CO')
mc_co_combined = combine_annual_and_stats(mc_co_annual_transaction_count, mc_co_stats, 'MC_CO')
mc_oc_combined = combine_annual_and_stats(mc_oc_annual_transaction_count, mc_oc_stats, 'MC_OC')

# Prepare the final DataFrame
final_df = pd.DataFrame()
final_df['Year'] = merged_mc_annual_transaction_count['Year']
final_df['Merged_MC'] = merged_mc_annual_transaction_count['Transaction_Count']
final_df['MC_CC'] = mc_cc_annual_transaction_count['Transaction_Count']
final_df['QQ_CC'] = qq_cc_annual_transaction_count['Transaction_Count']
final_df['QQ_CO'] = qq_co_annual_transaction_count['Transaction_Count']
final_df['MC_CO'] = mc_co_annual_transaction_count['Transaction_Count']
final_df['MC_OC'] = mc_oc_annual_transaction_count['Transaction_Count']

# Append statistics
stats_row = pd.DataFrame({
    'Year': ['$\\max$', '$\\min$', '$M$', '$\\mu$', '$\\sigma$'],
    'Merged_MC': [merged_mc_stats['Max'][0], merged_mc_stats['Min'][0], merged_mc_stats['Median'][0], merged_mc_stats['Mean'][0], merged_mc_stats['StdDev'][0]],
    'MC_CC': [mc_cc_stats['Max'][0], mc_cc_stats['Min'][0], mc_cc_stats['Median'][0], mc_cc_stats['Mean'][0], mc_cc_stats['StdDev'][0]],
    'QQ_CC': [qq_cc_stats['Max'][0], qq_cc_stats['Min'][0], qq_cc_stats['Median'][0], qq_cc_stats['Mean'][0], qq_cc_stats['StdDev'][0]],
    'QQ_CO': [qq_co_stats['Max'][0], qq_co_stats['Min'][0], qq_co_stats['Median'][0], qq_co_stats['Mean'][0], qq_co_stats['StdDev'][0]],
    'MC_CO': [mc_co_stats['Max'][0], mc_co_stats['Min'][0], mc_co_stats['Median'][0], mc_co_stats['Mean'][0], mc_co_stats['StdDev'][0]],
    'MC_OC': [mc_oc_stats['Max'][0], mc_oc_stats['Min'][0], mc_oc_stats['Median'][0], mc_oc_stats['Mean'][0], mc_oc_stats['StdDev'][0]]
})

final_df = pd.concat([final_df, stats_row], ignore_index=True)

# Save the combined results to a single CSV file
final_df.to_csv('/usr3/graduate/xz0224/all_strategies_transaction_analysis-little.csv', index=False)


# Strategy A

In [15]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'M1Y-Q3M': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_3m_mc_1y.xlsx',
    'M1Y-Q1Y': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_1y_mc_1y.xlsx',
    'M1Y': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_mc_1y_mc_1y.xlsx',
    'CO_1': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CC_1': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'OC_1': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CO_6': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx',  
    'CC_6': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'OC_6': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_6months_c-c).xlsx',
    'CO_3': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_3months_c-c).xlsx',  
    'CC_3': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_3months_c-c).xlsx',
    'OC_3': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_3months_c-c).xlsx'
}

strategies_data = {
    'S\&P': (buy_hold_dates, buy_hold_spy_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul6-mc-新内容', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul6-mc-新内容/end_year_values-jul6-mc-新内容.xlsx', index=False)
annual_returns_df.to_excel('jul6-mc-新内容/annual_returns-jul6-mc-新内容.xlsx', index=False)
max_drawdowns_df.to_excel('jul6-mc-新内容/max_drawdowns-jul6-mc-新内容.xlsx', index=False)
annual_volatility_df.to_excel('jul6-mc-新内容/annual_volatility-jul6-mc-新内容.xlsx', index=False)
sharpe_ratios_df.to_excel('jul6-mc-新内容/sharpe_ratios-jul6-mc-新内容.xlsx', index=False) 


/scratch/ipykernel_2478019/2139032807.py:48: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/2139032807.py:48: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/2139032807.py:48: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/2139032807.py:48: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/2139032807.py:48: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2478019/2139032807.py:48: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [1]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'M1Y': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_mc_1y_mc_1y.xlsx',
    'CO_1': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CC_1': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'OC_1': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CO_6': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx',  
    'CC_6': '/usr3/graduate/xz0224/normal/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx',
    'OC_6': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx',
}

strategies_data = {
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul15-mc-加强', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul15-mc-加强/end_year_values-jul15-mc-加强.xlsx', index=False)
annual_returns_df.to_excel('jul15-mc-加强/annual_returns-jul15-mc-加强.xlsx', index=False)
max_drawdowns_df.to_excel('jul15-mc-加强/max_drawdowns-jul15-mc-加强.xlsx', index=False)
annual_volatility_df.to_excel('jul15-mc-加强/annual_volatility-jul15-mc-加强.xlsx', index=False)
sharpe_ratios_df.to_excel('jul15-mc-加强/sharpe_ratios-jul15-mc-加强.xlsx', index=False) 


/scratch/ipykernel_3733015/1036951815.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/1036951815.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/1036951815.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/1036951815.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/1036951815.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/1036951815.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

# 超参数

In [5]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx'
}

strategies_data = {}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul20-spy-co', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul20-spy-co/end_year_values-jul20-spy-co.xlsx', index=False)
annual_returns_df.to_excel('jul20-spy-co/annual_returns-jul20-spy-co.xlsx', index=False)
max_drawdowns_df.to_excel('jul20-spy-co/max_drawdowns-jul20-spy-co.xlsx', index=False)
annual_volatility_df.to_excel('jul20-spy-co/annual_volatility-jul20-spy-co.xlsx', index=False)
sharpe_ratios_df.to_excel('jul20-spy-co/sharpe_ratios-jul20-spy-co.xlsx', index=False) 


/scratch/ipykernel_2239160/2950770481.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/2950770481.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/2950770481.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/2950770481.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/2950770481.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/2950770481.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [4]:
import pandas as pd
import empyrical
import os

file_paths = {
    'w10e10': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/normal/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx'
}

strategies_data = {}


# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul20-spy-cc', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul20-spy-cc/end_year_values-jul20-spy-cc.xlsx', index=False)
annual_returns_df.to_excel('jul20-spy-cc/annual_returns-jul20-spy-cc.xlsx', index=False)
max_drawdowns_df.to_excel('jul20-spy-cc/max_drawdowns-jul20-spy-cc.xlsx', index=False)
annual_volatility_df.to_excel('jul20-spy-cc/annual_volatility-jul20-spy-cc.xlsx', index=False)
sharpe_ratios_df.to_excel('jul20-spy-cc/sharpe_ratios-jul20-spy-cc.xlsx', index=False) 


/scratch/ipykernel_2239160/3475555628.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/3475555628.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/3475555628.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/3475555628.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/3475555628.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/3475555628.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [3]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/investment_results_final(2006)(train_3months_c-c)_加强.xlsx'
}

strategies_data = {}


# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul20-spy-oc', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul20-spy-oc/end_year_values-jul20-spy-oc.xlsx', index=False)
annual_returns_df.to_excel('jul20-spy-oc/annual_returns-jul20-spy-oc.xlsx', index=False)
max_drawdowns_df.to_excel('jul20-spy-oc/max_drawdowns-jul20-spy-oc.xlsx', index=False)
annual_volatility_df.to_excel('jul20-spy-oc/annual_volatility-jul20-spy-oc.xlsx', index=False)
sharpe_ratios_df.to_excel('jul20-spy-oc/sharpe_ratios-jul20-spy-oc.xlsx', index=False) 


/scratch/ipykernel_2239160/1879082453.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/1879082453.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/1879082453.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/1879082453.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/1879082453.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2239160/1879082453.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [9]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/mc_1year_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx'
}

strategies_data = {}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_max_drawdowns = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        max_drawdown = empyrical.max_drawdown(daily_returns)

        annual_max_drawdowns[year] = max_drawdown

    return year_end_values, annual_max_drawdowns

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, max_drawdowns = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply max drawdowns by 100 for percentage representation
max_drawdowns_df.iloc[:, 1:] *= 100
for df_name in ['end_year_values_df', 'max_drawdowns_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    # For end_year_values_df, set all summary stats to 'N/A'
    if df_name == 'end_year_values_df':
        for col in summary_stats.columns[1:]:
            summary_stats[col] = 'N/A'

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Function to generate LaTeX table from DataFrame
def df_to_latex_combined(val_df, mdd_df):
    # Creating the header with multirow and multicolumn
    header = '\\begin{table}[h!]\n\\centering\n\\caption{SPY-CO Different hyperparemters}\n\\begin{tabular}{c|' + 'cc||' * (val_df.shape[1] -  2) + 'cc}\n'
    header += '\\hline\n'
    header += '\\multirow{2}{*}{Year} '
    for col in val_df.columns[1:]:
        if col == val_df.columns[-1]:  # Last column without ||
            header += f'& \\multicolumn{{2}}{{c}}{{{col}}} '
        else:
            header += f'& \\multicolumn{{2}}{{c||}}{{{col}}} '
    header += '\\\\\cline{2-15}\n'
    header += ' & ' + ' & '.join(['val & mdd'] * (val_df.shape[1] - 1)) + ' \\\\\\hline\n'
    # Adding data rows
    body = ''
    for i in range(val_df.shape[0]):
        if i == val_df.shape[0] - 5:  # Before adding summary statistics
            body += '\\hline\\hline\n'
        row = [str(val_df.iloc[i, 0])]  # Add the year column
        for col in val_df.columns[1:]:
            val = val_df.at[i, col]
            mdd = mdd_df.at[i, col]
            row.append(str(int(val)) if val not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] and val != 'N/A' else val)
            row.append(f"{float(mdd):.1f}" if mdd not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] else mdd)
        body += ' & '.join(row) + '\\\\\n'
    
    # Footer
    footer = '\\hline\n\\end{tabular}\n\\end{table}'
    
    return header + body + footer

# Generate LaTeX tables
latex_table = df_to_latex_combined(end_year_values_df, max_drawdowns_df)

# Print LaTeX tables
print("Combined Table (End Year Values and Max Drawdowns):\n", latex_table)


Combined Table (End Year Values and Max Drawdowns):
 \begin{table}[h!]
\centering
\caption{SPY-CO Different hyperparemters}
\begin{tabular}{c|cc||cc||cc||cc||cc||cc||cc}
\hline
\multirow{2}{*}{Year} & \multicolumn{2}{c||}{w10e10} & \multicolumn{2}{c||}{w10e20} & \multicolumn{2}{c||}{w20e10} & \multicolumn{2}{c||}{w20e20} & \multicolumn{2}{c||}{w30e10} & \multicolumn{2}{c||}{w30e20} & \multicolumn{2}{c}{w30e20D} \\\cline{2-15}
 & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd \\\hline
2000 & 124 & -6.0 & 124 & -6.0 & 124 & -6.0 & 124 & -6.0 & 124 & -6.0 & 124 & -6.0 & 124 & -6.0\\
2001 & 120 & -7.6 & 113 & -14.3 & 127 & -7.1 & 114 & -13.8 & 124 & -5.7 & 128 & -5.9 & 115 & -13.3\\
2002 & 120 & -7.3 & 111 & -8.0 & 134 & -6.4 & 109 & -11.9 & 119 & -11.1 & 129 & -5.9 & 112 & -10.0\\
2003 & 126 & -2.2 & 109 & -4.6 & 135 & -2.3 & 112 & -5.1 & 123 & -2.2 & 133 & -3.7 & 116 & -2.4\\
2004 & 131 & -4.2 & 114 & -4.2 & 141 & -4.2 & 115 & -4.3 & 129 & -4.2 & 139 & 

/scratch/ipykernel_2305164/3022505109.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/3022505109.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/3022505109.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/3022505109.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/3022505109.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/3022505109.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [11]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/normal/加强_mc/mc_1year_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/normal/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx'
}

strategies_data = {}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_max_drawdowns = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        max_drawdown = empyrical.max_drawdown(daily_returns)

        annual_max_drawdowns[year] = max_drawdown

    return year_end_values, annual_max_drawdowns

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, max_drawdowns = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply max drawdowns by 100 for percentage representation
max_drawdowns_df.iloc[:, 1:] *= 100
for df_name in ['end_year_values_df', 'max_drawdowns_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    # For end_year_values_df, set all summary stats to 'N/A'
    if df_name == 'end_year_values_df':
        for col in summary_stats.columns[1:]:
            summary_stats[col] = 'N/A'

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Function to generate LaTeX table from DataFrame
def df_to_latex_combined(val_df, mdd_df):
    # Creating the header with multirow and multicolumn
    header = '\\begin{table}[h!]\n\\centering\n\\caption{SPY-CC Different hyperparemters}\n\\begin{tabular}{c|' + 'cc||' * (val_df.shape[1] -  2) + 'cc}\n'
    header += '\\hline\n'
    header += '\\multirow{2}{*}{Year} '
    for col in val_df.columns[1:]:
        if col == val_df.columns[-1]:  # Last column without ||
            header += f'& \\multicolumn{{2}}{{c}}{{{col}}} '
        else:
            header += f'& \\multicolumn{{2}}{{c||}}{{{col}}} '
    header += '\\\\\cline{2-15}\n'
    header += ' & ' + ' & '.join(['val & mdd'] * (val_df.shape[1] - 1)) + ' \\\\\\hline\n'
    # Adding data rows
    body = ''
    for i in range(val_df.shape[0]):
        if i == val_df.shape[0] - 5:  # Before adding summary statistics
            body += '\\hline\\hline\n'
        row = [str(val_df.iloc[i, 0])]  # Add the year column
        for col in val_df.columns[1:]:
            val = val_df.at[i, col]
            mdd = mdd_df.at[i, col]
            row.append(str(int(val)) if val not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] and val != 'N/A' else val)
            row.append(f"{float(mdd):.1f}" if mdd not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] else mdd)
        body += ' & '.join(row) + '\\\\\n'
    
    # Footer
    footer = '\\hline\n\\end{tabular}\n\\end{table}'
    
    return header + body + footer

# Generate LaTeX tables
latex_table = df_to_latex_combined(end_year_values_df, max_drawdowns_df)

# Print LaTeX tables
print("", latex_table)


 \begin{table}[h!]
\centering
\caption{SPY-CC Different hyperparemters}
\begin{tabular}{c|cc||cc||cc||cc||cc||cc||cc}
\hline
\multirow{2}{*}{Year} & \multicolumn{2}{c||}{w10e10} & \multicolumn{2}{c||}{w10e20} & \multicolumn{2}{c||}{w20e10} & \multicolumn{2}{c||}{w20e20} & \multicolumn{2}{c||}{w30e10} & \multicolumn{2}{c||}{w30e20} & \multicolumn{2}{c}{w30e20D} \\\cline{2-15}
 & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd \\\hline
2000 & 104 & -11.2 & 111 & -9.9 & 93 & -12.8 & 96 & -16.5 & 99 & -15.1 & 92 & -15.1 & 91 & -14.2\\
2001 & 83 & -21.3 & 93 & -21.3 & 83 & -18.7 & 82 & -17.4 & 84 & -20.3 & 92 & -19.5 & 84 & -22.3\\
2002 & 80 & -6.5 & 88 & -9.9 & 78 & -8.6 & 95 & -5.6 & 86 & -4.3 & 93 & -11.0 & 82 & -4.6\\
2003 & 89 & -4.4 & 101 & -5.2 & 86 & -5.2 & 105 & -5.7 & 99 & -4.9 & 112 & -4.4 & 94 & -5.2\\
2004 & 97 & -8.2 & 110 & -8.2 & 93 & -8.2 & 114 & -8.2 & 107 & -8.2 & 122 & -8.2 & 102 & -8.2\\
2005 & 100 & -7.3 & 113 & -7.3 & 96 & -7.3 & 119 

/scratch/ipykernel_2305164/4051222015.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4051222015.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4051222015.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4051222015.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4051222015.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4051222015.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [13]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_mc/mc_1year_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_mc/mc_1year_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_mc/mc_1year_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_mc/mc_1year_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_mc/mc_1year_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_mc/investment_results_final(2006)(train_1years_c-c)_加强.xlsx'
}

strategies_data = {}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_max_drawdowns = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        max_drawdown = empyrical.max_drawdown(daily_returns)

        annual_max_drawdowns[year] = max_drawdown

    return year_end_values, annual_max_drawdowns

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, max_drawdowns = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply max drawdowns by 100 for percentage representation
max_drawdowns_df.iloc[:, 1:] *= 100
for df_name in ['end_year_values_df', 'max_drawdowns_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    # For end_year_values_df, set all summary stats to 'N/A'
    if df_name == 'end_year_values_df':
        for col in summary_stats.columns[1:]:
            summary_stats[col] = 'N/A'

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Function to generate LaTeX table from DataFrame
def df_to_latex_combined(val_df, mdd_df):
    # Creating the header with multirow and multicolumn
    header = '\\begin{table}[h!]\n\\centering\n\\caption{Nasdaq-OC Different hyperparemters}\n\\begin{tabular}{c|' + 'cc||' * (val_df.shape[1] -  2) + 'cc}\n'
    header += '\\hline\n'
    header += '\\multirow{2}{*}{Year} '
    for col in val_df.columns[1:]:
        if col == val_df.columns[-1]:  # Last column without ||
            header += f'& \\multicolumn{{2}}{{c}}{{{col}}} '
        else:
            header += f'& \\multicolumn{{2}}{{c||}}{{{col}}} '
    header += '\\\\\cline{2-15}\n'
    header += ' & ' + ' & '.join(['val & mdd'] * (val_df.shape[1] - 1)) + ' \\\\\\hline\n'
    # Adding data rows
    body = ''
    for i in range(val_df.shape[0]):
        if i == val_df.shape[0] - 5:  # Before adding summary statistics
            body += '\\hline\\hline\n'
        row = [str(val_df.iloc[i, 0])]  # Add the year column
        for col in val_df.columns[1:]:
            val = val_df.at[i, col]
            mdd = mdd_df.at[i, col]
            row.append(str(int(val)) if val not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] and val != 'N/A' else val)
            row.append(f"{float(mdd):.1f}" if mdd not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] else mdd)
        body += ' & '.join(row) + '\\\\\n'
    
    # Footer
    footer = '\\hline\n\\end{tabular}\n\\end{table}'
    
    return header + body + footer

# Generate LaTeX tables
latex_table = df_to_latex_combined(end_year_values_df, max_drawdowns_df)

# Print LaTeX tables
print("Combined Table (End Year Values and Max Drawdowns):\n", latex_table)


Combined Table (End Year Values and Max Drawdowns):
 \begin{table}[h!]
\centering
\caption{SPY-OC Different hyperparemters}
\begin{tabular}{c|cc||cc||cc||cc||cc||cc||cc}
\hline
\multirow{2}{*}{Year} & \multicolumn{2}{c||}{w10e10} & \multicolumn{2}{c||}{w10e20} & \multicolumn{2}{c||}{w20e10} & \multicolumn{2}{c||}{w20e20} & \multicolumn{2}{c||}{w30e10} & \multicolumn{2}{c||}{w30e20} & \multicolumn{2}{c}{w30e20D} \\\cline{2-15}
 & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd \\\hline
2000 & 96 & -2.7 & 98 & -4.0 & 91 & -8.4 & 96 & -4.0 & 92 & -7.1 & 94 & -5.9 & 101 & -2.1\\
2001 & 96 & -3.0 & 102 & -3.6 & 99 & -3.8 & 105 & -3.6 & 96 & -3.6 & 99 & -3.6 & 96 & -7.6\\
2002 & 91 & -6.5 & 98 & -6.1 & 94 & -9.6 & 96 & -10.1 & 87 & -12.3 & 84 & -18.8 & 94 & -5.6\\
2003 & 108 & -6.9 & 108 & -10.6 & 107 & -8.5 & 105 & -8.1 & 105 & -9.7 & 92 & -7.1 & 101 & -11.0\\
2004 & 113 & -10.6 & 111 & -11.8 & 112 & -10.6 & 108 & -11.8 & 109 & -10.7 & 94 & -11.4 & 104 & -1

/scratch/ipykernel_2305164/639762099.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/639762099.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/639762099.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/639762099.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/639762099.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/639762099.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instea

In [17]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/normal/加强_qq/qqq_3_month_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/normal/加强_qq/qqq_3_month_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/normal/加强_qq/qqq_3_month_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/normal/加强_qq/qqq_3_month_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/normal/加强_qq/qqq_3_month_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/normal/加强_qq/investment_results_final(2006)(train_3months_c-c)_加强.xlsx'
}

strategies_data = {}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_max_drawdowns = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        max_drawdown = empyrical.max_drawdown(daily_returns)

        annual_max_drawdowns[year] = max_drawdown

    return year_end_values, annual_max_drawdowns

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, max_drawdowns = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply max drawdowns by 100 for percentage representation
max_drawdowns_df.iloc[:, 1:] *= 100
for df_name in ['end_year_values_df', 'max_drawdowns_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    # For end_year_values_df, set all summary stats to 'N/A'
    if df_name == 'end_year_values_df':
        for col in summary_stats.columns[1:]:
            summary_stats[col] = 'N/A'

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Function to generate LaTeX table from DataFrame
def df_to_latex_combined(val_df, mdd_df):
    # Creating the header with multirow and multicolumn
    header = '\\begin{table}[h!]\n\\centering\n\\caption{Nasdaq-CC Different hyperparemters}\n\\begin{tabular}{c|' + 'cc||' * (val_df.shape[1] -  2) + 'cc}\n'
    header += '\\hline\n'
    header += '\\multirow{2}{*}{Year} '
    for col in val_df.columns[1:]:
        if col == val_df.columns[-1]:  # Last column without ||
            header += f'& \\multicolumn{{2}}{{c}}{{{col}}} '
        else:
            header += f'& \\multicolumn{{2}}{{c||}}{{{col}}} '
    header += '\\\\\cline{2-15}\n'
    header += ' & ' + ' & '.join(['val & mdd'] * (val_df.shape[1] - 1)) + ' \\\\\\hline\n'
    # Adding data rows
    body = ''
    for i in range(val_df.shape[0]):
        if i == val_df.shape[0] - 5:  # Before adding summary statistics
            body += '\\hline\\hline\n'
        row = [str(val_df.iloc[i, 0])]  # Add the year column
        for col in val_df.columns[1:]:
            val = val_df.at[i, col]
            mdd = mdd_df.at[i, col]
            row.append(str(int(val)) if val not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] and val != 'N/A' else val)
            row.append(f"{float(mdd):.1f}" if mdd not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] else mdd)
        body += ' & '.join(row) + '\\\\\n'
    
    # Footer
    footer = '\\hline\n\\end{tabular}\n\\end{table}'
    
    return header + body + footer

# Generate LaTeX tables
latex_table = df_to_latex_combined(end_year_values_df, max_drawdowns_df)

# Print LaTeX tables
print("Combined Table (End Year Values and Max Drawdowns):\n", latex_table)


Combined Table (End Year Values and Max Drawdowns):
 \begin{table}[h!]
\centering
\caption{Nasdaq-OC Different hyperparemters}
\begin{tabular}{c|cc||cc||cc||cc||cc||cc||cc}
\hline
\multirow{2}{*}{Year} & \multicolumn{2}{c||}{w10e10} & \multicolumn{2}{c||}{w10e20} & \multicolumn{2}{c||}{w20e10} & \multicolumn{2}{c||}{w20e20} & \multicolumn{2}{c||}{w30e10} & \multicolumn{2}{c||}{w30e20} & \multicolumn{2}{c}{w30e20D} \\\cline{2-15}
 & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd \\\hline
2000 & 70 & -45.2 & 72 & -44.2 & 68 & -47.0 & 70 & -45.2 & 72 & -44.3 & 69 & -50.7 & 86 & -41.5\\
2001 & 74 & -11.4 & 63 & -21.2 & 58 & -24.2 & 75 & -11.4 & 60 & -21.4 & 57 & -22.5 & 72 & -18.3\\
2002 & 63 & -21.4 & 58 & -19.7 & 49 & -20.8 & 66 & -19.5 & 46 & -28.6 & 47 & -22.1 & 62 & -20.7\\
2003 & 81 & -18.5 & 75 & -19.4 & 68 & -12.5 & 88 & -16.4 & 58 & -18.9 & 61 & -18.6 & 81 & -18.4\\
2004 & 91 & -14.6 & 84 & -12.9 & 74 & -14.4 & 101 & -10.8 & 58 & -17.5 & 67 & -13

/scratch/ipykernel_2305164/785398433.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/785398433.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/785398433.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/785398433.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/785398433.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/785398433.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instea

In [19]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',
    'w10e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_qq/qqq_3_month_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_qq/qqq_3_month_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_qq/qqq_3_month_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_qq/qqq_3_month_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_qq/qqq_3_month_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_qq/investment_results_final(2006)(train_3months_c-c)_加强.xlsx'
}

strategies_data = {}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_max_drawdowns = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        max_drawdown = empyrical.max_drawdown(daily_returns)

        annual_max_drawdowns[year] = max_drawdown

    return year_end_values, annual_max_drawdowns

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, max_drawdowns = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply max drawdowns by 100 for percentage representation
max_drawdowns_df.iloc[:, 1:] *= 100
for df_name in ['end_year_values_df', 'max_drawdowns_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    # For end_year_values_df, set all summary stats to 'N/A'
    if df_name == 'end_year_values_df':
        for col in summary_stats.columns[1:]:
            summary_stats[col] = 'N/A'

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Function to generate LaTeX table from DataFrame
def df_to_latex_combined(val_df, mdd_df):
    # Creating the header with multirow and multicolumn
    header = '\\begin{table}[h!]\n\\centering\n\\caption{Nasdaq-CO Different hyperparemters}\n\\begin{tabular}{c|' + 'cc||' * (val_df.shape[1] -  2) + 'cc}\n'
    header += '\\hline\n'
    header += '\\multirow{2}{*}{Year} '
    for col in val_df.columns[1:]:
        if col == val_df.columns[-1]:  # Last column without ||
            header += f'& \\multicolumn{{2}}{{c}}{{{col}}} '
        else:
            header += f'& \\multicolumn{{2}}{{c||}}{{{col}}} '
    header += '\\\\\cline{2-15}\n'
    header += ' & ' + ' & '.join(['val & mdd'] * (val_df.shape[1] - 1)) + ' \\\\\\hline\n'
    # Adding data rows
    body = ''
    for i in range(val_df.shape[0]):
        if i == val_df.shape[0] - 5:  # Before adding summary statistics
            body += '\\hline\\hline\n'
        row = [str(val_df.iloc[i, 0])]  # Add the year column
        for col in val_df.columns[1:]:
            val = val_df.at[i, col]
            mdd = mdd_df.at[i, col]
            row.append(str(int(val)) if val not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] and val != 'N/A' else val)
            row.append(f"{float(mdd):.1f}" if mdd not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] else mdd)
        body += ' & '.join(row) + '\\\\\n'
    
    # Footer
    footer = '\\hline\n\\end{tabular}\n\\end{table}'
    
    return header + body + footer

# Generate LaTeX tables
latex_table = df_to_latex_combined(end_year_values_df, max_drawdowns_df)

# Print LaTeX tables
print("Combined Table (End Year Values and Max Drawdowns):\n", latex_table)


Combined Table (End Year Values and Max Drawdowns):
 \begin{table}[h!]
\centering
\caption{Nasdaq-CO Different hyperparemters}
\begin{tabular}{c|cc||cc||cc||cc||cc||cc||cc}
\hline
\multirow{2}{*}{Year} & \multicolumn{2}{c||}{w10e10} & \multicolumn{2}{c||}{w10e20} & \multicolumn{2}{c||}{w20e10} & \multicolumn{2}{c||}{w20e20} & \multicolumn{2}{c||}{w30e10} & \multicolumn{2}{c||}{w30e20} & \multicolumn{2}{c}{w30e20D} \\\cline{2-15}
 & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd \\\hline
2000 & 128 & -14.9 & 129 & -14.9 & 117 & -14.9 & 118 & -14.9 & 129 & -14.9 & 116 & -14.9 & 139 & -14.9\\
2001 & 141 & -8.0 & 131 & -8.0 & 113 & -8.0 & 134 & -8.0 & 131 & -7.7 & 109 & -9.0 & 150 & -8.0\\
2002 & 144 & -9.9 & 128 & -9.9 & 120 & -9.9 & 141 & -9.9 & 139 & -9.9 & 112 & -9.9 & 147 & -9.9\\
2003 & 179 & -6.9 & 159 & -7.4 & 146 & -7.3 & 170 & -7.5 & 168 & -7.1 & 138 & -5.9 & 179 & -8.9\\
2004 & 208 & -2.6 & 186 & -2.6 & 166 & -3.1 & 186 & -4.6 & 189 & -4.6 & 15

/scratch/ipykernel_2305164/4269503100.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4269503100.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4269503100.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4269503100.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4269503100.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/4269503100.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [20]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'w10e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'w10e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w10_e20.xlsx',
    'w20e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w20_e10.xlsx',
    'w20e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w20_e20.xlsx',
    'w30e10': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w30_e10.xlsx',
    'w30e20': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/qqq_3_month_w30_e20.xlsx',
    'w30e20D': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/investment_results_final(2006)(train_3months_c-c)_加强.xlsx'
}

strategies_data = {}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_max_drawdowns = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        max_drawdown = empyrical.max_drawdown(daily_returns)

        annual_max_drawdowns[year] = max_drawdown

    return year_end_values, annual_max_drawdowns

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, max_drawdowns = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply max drawdowns by 100 for percentage representation
max_drawdowns_df.iloc[:, 1:] *= 100
for df_name in ['end_year_values_df', 'max_drawdowns_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    # For end_year_values_df, set all summary stats to 'N/A'
    if df_name == 'end_year_values_df':
        for col in summary_stats.columns[1:]:
            summary_stats[col] = 'N/A'

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Function to generate LaTeX table from DataFrame
def df_to_latex_combined(val_df, mdd_df):
    # Creating the header with multirow and multicolumn
    header = '\\begin{table}[h!]\n\\centering\n\\caption{Nasdaq-OC Different hyperparemters}\n\\begin{tabular}{c|' + 'cc||' * (val_df.shape[1] -  2) + 'cc}\n'
    header += '\\hline\n'
    header += '\\multirow{2}{*}{Year} '
    for col in val_df.columns[1:]:
        if col == val_df.columns[-1]:  # Last column without ||
            header += f'& \\multicolumn{{2}}{{c}}{{{col}}} '
        else:
            header += f'& \\multicolumn{{2}}{{c||}}{{{col}}} '
    header += '\\\\\cline{2-15}\n'
    header += ' & ' + ' & '.join(['val & mdd'] * (val_df.shape[1] - 1)) + ' \\\\\\hline\n'
    # Adding data rows
    body = ''
    for i in range(val_df.shape[0]):
        if i == val_df.shape[0] - 5:  # Before adding summary statistics
            body += '\\hline\\hline\n'
        row = [str(val_df.iloc[i, 0])]  # Add the year column
        for col in val_df.columns[1:]:
            val = val_df.at[i, col]
            mdd = mdd_df.at[i, col]
            row.append(str(int(val)) if val not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] and val != 'N/A' else val)
            row.append(f"{float(mdd):.1f}" if mdd not in [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'] else mdd)
        body += ' & '.join(row) + '\\\\\n'
    
    # Footer
    footer = '\\hline\n\\end{tabular}\n\\end{table}'
    
    return header + body + footer

# Generate LaTeX tables
latex_table = df_to_latex_combined(end_year_values_df, max_drawdowns_df)

# Print LaTeX tables
print("Combined Table (End Year Values and Max Drawdowns):\n", latex_table)


Combined Table (End Year Values and Max Drawdowns):
 \begin{table}[h!]
\centering
\caption{Nasdaq-OC Different hyperparemters}
\begin{tabular}{c|cc||cc||cc||cc||cc||cc||cc}
\hline
\multirow{2}{*}{Year} & \multicolumn{2}{c||}{w10e10} & \multicolumn{2}{c||}{w10e20} & \multicolumn{2}{c||}{w20e10} & \multicolumn{2}{c||}{w20e20} & \multicolumn{2}{c||}{w30e10} & \multicolumn{2}{c||}{w30e20} & \multicolumn{2}{c}{w30e20D} \\\cline{2-15}
 & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd & val & mdd \\\hline
2000 & 48 & -51.4 & 53 & -48.1 & 48 & -53.8 & 54 & -48.3 & 52 & -49.9 & 58 & -40.6 & 55 & -46.4\\
2001 & 51 & -9.5 & 58 & -9.5 & 51 & -9.5 & 53 & -15.5 & 62 & -8.3 & 60 & -12.7 & 57 & -11.9\\
2002 & 45 & -17.0 & 41 & -32.2 & 48 & -17.7 & 39 & -30.2 & 56 & -17.8 & 46 & -27.0 & 44 & -25.7\\
2003 & 45 & -20.5 & 42 & -19.7 & 50 & -20.1 & 40 & -16.9 & 57 & -15.9 & 48 & -17.7 & 44 & -20.1\\
2004 & 41 & -21.1 & 35 & -27.6 & 46 & -21.6 & 35 & -25.3 & 52 & -22.9 & 43 & -22.5 & 

/scratch/ipykernel_2305164/2404998184.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/2404998184.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/2404998184.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/2404998184.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/2404998184.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2305164/2404998184.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

# Strategy B

In [ ]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'M1Y-Q3M': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_3m_mc_1y.xlsx',
    'M1Y-Q1Y': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_1y_mc_1y.xlsx',
    'CO_1': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CC_1': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'OC_1': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CO_6': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 6月很高.xlsx',  
    'CC_6': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'OC_6': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'CO_3': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',  
    'CC_3': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'OC_3': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx'
}

strategies_data = {
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul7-qq-新内容', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul7-qq-新内容/end_year_values-jul7-qq-新内容.xlsx', index=False)
annual_returns_df.to_excel('jul7-qq-新内容/annual_returns-jul7-qq-新内容.xlsx', index=False)
max_drawdowns_df.to_excel('jul7-qq-新内容/max_drawdowns-jul7-qq-新内容.xlsx', index=False)
annual_volatility_df.to_excel('jul7-qq-新内容/annual_volatility-jul7-qq-新内容.xlsx', index=False)
sharpe_ratios_df.to_excel('jul7-qq-新内容/sharpe_ratios-jul7-qq-新内容.xlsx', index=False) 


In [ ]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'M1Y-Q3M': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_3m_mc_1y.xlsx',
    'M1Y-Q1Y': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_1y_mc_1y.xlsx',
    'CO_1': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CC_1': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'OC_1': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CO_6': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 6月很高.xlsx',  
    'CC_6': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'OC_6': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'CO_3': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',  
    'CC_3': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'OC_3': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx'
}

strategies_data = {
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul7-qq-新内容', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul7-qq-新内容/end_year_values-jul7-qq-新内容.xlsx', index=False)
annual_returns_df.to_excel('jul7-qq-新内容/annual_returns-jul7-qq-新内容.xlsx', index=False)
max_drawdowns_df.to_excel('jul7-qq-新内容/max_drawdowns-jul7-qq-新内容.xlsx', index=False)
annual_volatility_df.to_excel('jul7-qq-新内容/annual_volatility-jul7-qq-新内容.xlsx', index=False)
sharpe_ratios_df.to_excel('jul7-qq-新内容/sharpe_ratios-jul7-qq-新内容.xlsx', index=False) 


In [3]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'CO_1': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CC_1': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'OC_1': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CO_6': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 6月很高.xlsx',  
    'CC_6': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'OC_6': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'CO_3': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',  
    'CC_3': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'OC_3': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx'
}

strategies_data = {
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul9', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul9/end_year_values-jul9.xlsx', index=False)
annual_returns_df.to_excel('jul9/annual_returns-jul9.xlsx', index=False)
max_drawdowns_df.to_excel('jul9/max_drawdowns-jul9.xlsx', index=False)
annual_volatility_df.to_excel('jul9/annual_volatility-jul9.xlsx', index=False)
sharpe_ratios_df.to_excel('jul9/sharpe_ratios-jul9.xlsx', index=False) 


/scratch/ipykernel_2103583/987953100.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2103583/987953100.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2103583/987953100.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2103583/987953100.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2103583/987953100.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2103583/987953100.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instea

In [3]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'CO_3': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',  
    'CC_3': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'OC_3': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'CO_6': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/加强_qq/investment_results_final(2006)(train_3months_c-c)_加强.xlsx',  
    'CC_6': '/usr3/graduate/xz0224/normal/加强_qq/investment_results_final(2006)(train_3months_c-c)_加强.xlsx',
    'OC_6': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/加强_qq/investment_results_final(2006)(train_3months_c-c)_加强.xlsx',
}

strategies_data = {
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul15-qq-加强', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul15-qq-加强/end_year_values-jul15-qq-加强.xlsx', index=False)
annual_returns_df.to_excel('jul15-qq-加强/annual_returns-jul15-qq-加强.xlsx', index=False)
max_drawdowns_df.to_excel('jul15-qq-加强/max_drawdowns-jul15-qq-加强.xlsx', index=False)
annual_volatility_df.to_excel('jul15-qq-加强/annual_volatility-jul15-qq-加强.xlsx', index=False)
sharpe_ratios_df.to_excel('jul15-qq-加强/sharpe_ratios-jul15-qq-加强.xlsx', index=False) 


/scratch/ipykernel_3733015/2920564024.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/2920564024.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/2920564024.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/2920564024.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/2920564024.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_3733015/2920564024.py:41: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' 

In [3]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'B': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',  
    'CO_1': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CC_1': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'OC_1': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/investment_results_final(2006)(train_1years_c-c).xlsx',
    'CO_6': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 6月很高.xlsx',  
    'CC_6': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'OC_6': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'CC_3': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'OC_3': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx'
}

strategies_data = {
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul11-B', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul11-B/end_year_values-jul11-B.xlsx', index=False)
annual_returns_df.to_excel('jul11-B/annual_returns-jul11-B.xlsx', index=False)
max_drawdowns_df.to_excel('jul11-B/max_drawdowns-jul11-B.xlsx', index=False)
annual_volatility_df.to_excel('jul11-B/annual_volatility-jul11-B.xlsx', index=False)
sharpe_ratios_df.to_excel('jul11-B/sharpe_ratios-jul11-B.xlsx', index=False) 


/scratch/ipykernel_2774620/614141822.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2774620/614141822.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2774620/614141822.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2774620/614141822.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2774620/614141822.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2774620/614141822.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instea

In [ ]:
# 读取Excel文件中的数据路径
file_paths = {
    'QQ-1Y-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_1years_c-c).xlsx',
    'QQ-1Y-CC': '/usr3/graduate/xz0224/normal/investment_results_final(2006)(train_1years_c-c).xlsx',    
    'QQ-1Y-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/investment_results_final(2006)(train_1years_c-c).xlsx',
    'QQ-6M-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 6月很高.xlsx',  
    'QQ-6M-CC': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'QQ-6M-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_6month_c-c).xlsx',
    'QQ-3M-CO': '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx',  
    'QQ-3M-CC': '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
    'QQ-3M-OC': '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx',
}



# Strategy C

In [5]:
import pandas as pd
import empyrical
import os

# 读取Excel文件中的数据路径
file_paths = {
    'M1Y-Q3M': '/usr3/graduate/xz0224/fused(融合策略)/fused_strategy_results_qq_3m_mc_1y.xlsx'
}

strategies_data = {
    'S\&P': (buy_hold_dates, buy_hold_spy_values),
    'BH-QQ': (buy_hold_dates, buy_hold_qqq_cc_values)
}

# Function to read data from Excel or CSV files
def read_data(file_path):
    if file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    return pd.to_datetime(data['Date']), data['Investment_Value']

# Read the data from the files and add to strategies_data
for name, path in file_paths.items():
    strategies_data[name] = read_data(path)

# Initialize DataFrames to store results
end_year_values_df = pd.DataFrame()
annual_returns_df = pd.DataFrame()
max_drawdowns_df = pd.DataFrame()
annual_volatility_df = pd.DataFrame()
sharpe_ratios_df = pd.DataFrame()

# Function to analyze investment strategy
def analyze_investment_strategy(dates, values):
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'value': values})
    df.set_index('date', inplace=True)
    grouped = df.resample('Y')
    year_end_values = {}
    annual_returns = {}
    annual_max_drawdowns = {}
    annual_volatility = {}
    sharpe_ratios = {}

    for name, group in grouped:
        year = name.year
        year_end_values[year] = group.iloc[-1]['value']
        daily_returns = empyrical.simple_returns(group['value'])
        annual_return = empyrical.annual_return(daily_returns)
        max_drawdown = empyrical.max_drawdown(daily_returns)
        volatility = empyrical.annual_volatility(daily_returns)
        sharpe_ratio = annual_return / volatility if volatility != 0 else 0

        annual_returns[year] = annual_return
        annual_max_drawdowns[year] = max_drawdown
        annual_volatility[year] = volatility
        sharpe_ratios[year] = sharpe_ratio

    return year_end_values, annual_returns, annual_max_drawdowns, annual_volatility, sharpe_ratios

# Analyze each strategy and collect results
for strategy_name, (strategy_dates, strategy_values) in strategies_data.items():
    year_end_values, annual_returns, max_drawdowns, annual_volatility, sharpe_ratios = analyze_investment_strategy(strategy_dates, strategy_values)
    year_end_values_df_ = pd.DataFrame.from_dict(year_end_values, orient='index', columns=[strategy_name])
    annual_returns_df_ = pd.DataFrame.from_dict(annual_returns, orient='index', columns=[strategy_name])
    max_drawdowns_df_ = pd.DataFrame.from_dict(max_drawdowns, orient='index', columns=[strategy_name])
    annual_volatility_df_ = pd.DataFrame.from_dict(annual_volatility, orient='index', columns=[strategy_name])
    sharpe_ratios_df_ = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=[strategy_name])

    end_year_values_df = pd.concat([end_year_values_df, year_end_values_df_], axis=1)
    annual_returns_df = pd.concat([annual_returns_df, annual_returns_df_], axis=1)
    max_drawdowns_df = pd.concat([max_drawdowns_df, max_drawdowns_df_], axis=1)
    annual_volatility_df = pd.concat([annual_volatility_df, annual_volatility_df_], axis=1)
    sharpe_ratios_df = pd.concat([sharpe_ratios_df, sharpe_ratios_df_], axis=1)

# Reset index to add 'Year' as the first column
end_year_values_df.reset_index(inplace=True)
annual_returns_df.reset_index(inplace=True)
max_drawdowns_df.reset_index(inplace=True)
annual_volatility_df.reset_index(inplace=True)
sharpe_ratios_df.reset_index(inplace=True)

# Rename the index column to 'Year'
end_year_values_df.rename(columns={'index': 'Year'}, inplace=True)
annual_returns_df.rename(columns={'index': 'Year'}, inplace=True)
max_drawdowns_df.rename(columns={'index': 'Year'}, inplace=True)
annual_volatility_df.rename(columns={'index': 'Year'}, inplace=True)
sharpe_ratios_df.rename(columns={'index': 'Year'}, inplace=True)

# Multiply the results by 100 for percentage representation
annual_returns_df.iloc[:, 1:] *= 100
max_drawdowns_df.iloc[:, 1:] *= 100
annual_volatility_df.iloc[:, 1:] *= 100

# Calculate summary statistics and append them as a new row for each strategy
for df_name in ['annual_returns_df', 'max_drawdowns_df', 'annual_volatility_df', 'sharpe_ratios_df']:
    df = locals()[df_name]
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()

    summary_stats = pd.DataFrame({
        'Year': [r'$\max$', r'$\min$', r'$M$', r'$\mu$', r'$\sigma$'],
        **{col: [max_values[col], min_values[col], median_values[col], average_values[col], stddev_values[col]] for col in df.columns[1:]}
    })
    
    if df_name == 'max_drawdowns_df':  # For max drawdowns, max is min and min is max
        summary_stats.iloc[0, 1:], summary_stats.iloc[1, 1:] = summary_stats.iloc[1, 1:].values, summary_stats.iloc[0, 1:].values

    updated_df = pd.concat([df, summary_stats], ignore_index=True)
    locals()[df_name] = updated_df

# Create the directory if it does not exist
os.makedirs('jul9-part3', exist_ok=True)

# Define a function to rename columns based on strategy type
def rename_columns(df):
    columns = df.columns
    new_columns = ['Year']
    for col in columns[1:]:
        if 'CO' in col:
            new_columns.append('CO')
        elif 'CC' in col:
            new_columns.append('CC')
        elif 'OC' in col:
            new_columns.append('OC')
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Rename columns before saving
end_year_values_df = rename_columns(end_year_values_df)
annual_returns_df = rename_columns(annual_returns_df)
max_drawdowns_df = rename_columns(max_drawdowns_df)
annual_volatility_df = rename_columns(annual_volatility_df)
sharpe_ratios_df = rename_columns(sharpe_ratios_df)

# Save DataFrames to .xlsx files
end_year_values_df.to_excel('jul9-part3/end_year_values-jul9-part3.xlsx', index=False)
annual_returns_df.to_excel('jul9-part3/annual_returns-jul9-part3.xlsx', index=False)
max_drawdowns_df.to_excel('jul9-part3/max_drawdowns-jul9-part3.xlsx', index=False)
annual_volatility_df.to_excel('jul9-part3/annual_volatility-jul9-part3.xlsx', index=False)
sharpe_ratios_df.to_excel('jul9-part3/sharpe_ratios-jul9-part3.xlsx', index=False) 


/scratch/ipykernel_2103583/957391040.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2103583/957391040.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
/scratch/ipykernel_2103583/957391040.py:38: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  grouped = df.resample('Y')
